# Transfer learning _ResNet-50

# 1.Importing libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

# 2.Loading data/ Preprocessing

In [2]:
data_dir = "dermnet/train"#directory of the  train data
test_dir = "dermnet/test"#directory of the  test data

In [4]:
img_height,img_width=180,180#Size of the image
batch_size=32
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),# resize images after they are read from folder
  batch_size=batch_size)#Size of the batches of data to load at a time

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'dermnet/train'

In [4]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 24855 files belonging to 23 classes.
Using 4971 files for validation.


In [6]:
#testing data preprocessing
test_ds =tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,#directory of the  test data
    shuffle = True,
    seed=123,
    image_size =(img_height, img_width),
    batch_size = 32  
)

Found 4317 files belonging to 23 classes.


Data visualization

In [ ]:
class_names=train_ds.class_names
print(class_names)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(6):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
resnet_model = Sequential()


pretrained_model= tf.keras.applications.ResNet50(include_top=,
                   input_shape=(180,180,3),
                   pooling='avg',classes=4,
                   weights='imagenet')

#this is test 